<a href="https://colab.research.google.com/github/paulf35/cd-ds-productsalespredictions/blob/main/CD_Paul_Foy_Project_1_Part_5_ProductSalesPredictions_Core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Science - Project 1 - Part 5 (CORE)
Author: Paul Foy

We will continue to work on your sales prediction project. The goal of this step is to help the retailer by using machine learning to make predictions about future sales based on the data provided.

For Part 5, you will go back to your original, uncleaned, sales prediction dataset with the goal of preventing data leakage.

You should load a fresh version of the original data set here using pd.read_csv() and start your cleaning process over to ensure there is no data leakage!

- Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)
- Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.
- Perform a train test split
- Create a preprocessing object to prepare the dataset for Machine Learning
- Make sure your imputation of missing values occurs after the train test split using SimpleImputer.

#Import required libraries

In [221]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from sklearn.pipeline import make_pipeline

from sklearn import set_config
set_config(display='diagram')

# Warnings
import warnings

## Set filter warnings to ignore
warnings.filterwarnings('ignore')

# Set the default transformation output to Pandas
from sklearn import set_config
set_config(transform_output='pandas')

#Pandas Display Configurations


In [222]:
## Display all columns
pd.set_option('display.max_column', None)

#Load the data

In [223]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [224]:
# Load Sales data in from Google Drive
fname = "/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv"
df = pd.read_csv(fname)

#Inspect and Fix Data Issues


In [225]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [226]:
# Make a copy of original df to avoid any manipulations
eda_ml = df.copy()

#Check for duplicated, Missing , or Inconsistent data

In [227]:
# Check to see if there are any duplicate rows
eda_ml.duplicated().sum()

0

In [228]:
# Display the sum of missing values
eda_ml.isna().sum().sum()

3873

In [229]:
# Display desriptive statitistics for all collumns
eda_ml.describe(include='number')

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [230]:
# Display desriptive statitistics for all collumns
eda_ml.describe(exclude='number')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


##Fix data inconsistencies

Inconsistencies found and fixed:
- Item_Fat_Content
  - "LF" and "low fat" changed to "Low Fat"
  - "reg" changed to "Regular"
- Outlet_Size
  - "High" changed to "Large"

In [231]:
for col in eda_ml:
  print(f'Count for {col} is: \n{eda_ml[col].value_counts()}\n')

Count for Item_Identifier is: 
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

Count for Item_Weight is: 
12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64

Count for Item_Fat_Content is: 
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

Count for Item_Visibility is: 
0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64

Count for Item_Type is: 
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy            

In [232]:
# Fix inconsistencies in Item_Fat_Content
#Display the value counts from the column
eda_ml['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [233]:
# Use a Dictionary to replace values
eda_ml.replace({'low fat': 'Low Fat', 'reg':'Regular','LF':'Low Fat', 'High':'Large'}, inplace = True)

In [234]:
#Display the value counts from the column
print(eda_ml['Item_Fat_Content'].value_counts())

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64


In [235]:
#Display the value counts from Outlet_Size
print(eda_ml['Outlet_Size'].value_counts())

Medium    2793
Small     2388
Large      932
Name: Outlet_Size, dtype: int64


#Determine Ordinal features
Ordinal features: Outlet_Size, Item_Fat_Content, Outlet_Location_Type

Check ordinal categories

In [236]:
eda_ml['Outlet_Size'].value_counts()

Medium    2793
Small     2388
Large      932
Name: Outlet_Size, dtype: int64

In [237]:
eda_ml['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [238]:
eda_ml['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

#Split the Data (Validation Split)

In [239]:
# split X and y, we are predicting price
target = 'Item_Outlet_Sales'
X = eda_ml.drop(columns=[target]).copy()
y = eda_ml[target].copy()

# split training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30 , random_state=42)

In [240]:
X_train.shape

(5966, 11)

In [241]:
X_test.shape

(2557, 11)

In [242]:
X_train.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
dtype: object

#Create Pipelins and Tuples for Group of Columns

Pipeline types:
- Numeric: Item_Weight, Item_Fat_Content, Item_Visibility, Item_MRP, Outlet_Establishment_Year
- Nominal: Item_Identifier, Item_Type, Outlet_Type, Outlet_Identifier, Outlet_Establishment_Year
- Ordinal: Outlet_Size, Item_Fat_Content, Outlet_Location_Type

##Numeric

In [243]:
# PREPROCESSING PIPELINE FOR NUMERIC DATA

# Save list of number column names
num_cols = X_train.select_dtypes('number').columns

# Transformers
scaler = StandardScaler()
med_imputer = SimpleImputer(strategy = 'median')

# Pipeline
num_pipeline = make_pipeline(med_imputer, scaler)
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [244]:
# Numeric Tuple
numeric_tuple = ('number',num_pipeline, num_cols)

#Ordinal

In [245]:
# PREPROCESSING PIPELINE FOR ORDINAL DATA

# Save list of number column names
ordinal_cols = ['Outlet_Size', 'Item_Fat_Content','Outlet_Location_Type']

# Ordered Category Lists

Outlet_Size_list = ['Small','Medium','Large']
Item_Fat_Content_list = ['Low Fat','Regular']
Outlet_Location_Type_list = ['Tier 1','Tier 2','Tier 3']

# Transformers

ord = OrdinalEncoder( categories = [Outlet_Size_list, Item_Fat_Content_list, Outlet_Location_Type_list])
freq_imputer = SimpleImputer(strategy='most_frequent')

# you might have 100 diff cat for ordinal so its getting out of range so good to scale
scaler2 = StandardScaler()

# Pipeline
ord_pipeline = make_pipeline(freq_imputer, ord, scaler2)

ord_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Small', 'Medium', 'Large'],
                                            ['Low Fat', 'Regular'],
                                            ['Tier 1', 'Tier 2', 'Tier 3']])),
                ('standardscaler', StandardScaler())])

In [246]:
#Ordinal Tuple
ord_tuple = ('ordinal',ord_pipeline, ordinal_cols)

#Nominal


In [247]:
# PREPROCESSING PIPELINE FOR ONE-HOT-ENCODED DATA

# Save list of nominal column names
nominal_cols = X_train.select_dtypes('object').drop(columns=ordinal_cols).columns

# Transformers

missing_imputer = SimpleImputer(strategy='constant', fill_value='missing')
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Pipeline
nom_pipeline = make_pipeline(missing_imputer , ohe)
nom_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [248]:
# Nominal Tuple
ohe_tuple = ('categorical',nom_pipeline, nominal_cols)

#Create Column Transformer to Apply Different Preprocessing to Different Columns

In [249]:
# Instantiate the make column transformer
col_transformer = ColumnTransformer([numeric_tuple,
                                       ord_tuple,
                                       ohe_tuple],
                                       remainder='drop', verbose_feature_names_out=False)

#Fit the Column Transformer on the Training Data Only

In [250]:
# Fit the column transformer on the X_train
col_transformer.fit(X_train)

ColumnTransformer(transformers=[('number',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  Ordi...
                                 ['Outlet_Size', 'Item_Fat_Content',
                                  'Outlet_Location_Type']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'], dtype='object'))],
                  verbose_feature_names_out=False)

# Transform Both Training and Testing Data

In [251]:
# Transform the X_train and the X_test
X_train_proc = col_transformer.transform(X_train)
X_test_proc = col_transformer.transform(X_test)

#Check the Result

In [252]:
# Display the first (5) rows of the dataframe
display(X_train_proc.head())
# Check the shape
print(f'\nshape of processed data is: {X_train_proc.shape}')
# Check for remaining missing values
print(f'\nThere are {X_train_proc.isna().sum().sum()} missing values')
# Check the data types
print(f'\nThe datatypes are {X_train_proc.dtypes}')

Item_Weight  Item_Visibility  Item_MRP  Outlet_Establishment_Year  \
1921    -1.225867         1.606681 -0.401895                   0.015833   
655      1.627719        -1.008442 -0.619284                  -0.103437   
2229    -0.056443        -0.227068 -0.202535                  -1.534680   
2537     1.532837         1.028038  1.290187                   0.492914   
3528     1.295631         0.515355  0.686853                  -1.296139   

      Outlet_Size  Item_Fat_Content  Outlet_Location_Type  \
1921     0.284752          1.353831              1.083627   
655     -1.384039         -0.738645             -1.383428   
2229     0.284752          1.353831              1.083627   
2537     0.284752         -0.738645             -0.149901   
3528     1.953543          1.353831              1.083627   

      Item_Identifier_DRA12  Item_Identifier_DRA24  Item_Identifier_DRA59  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRB01  Item_Identifier_DRB13  Item_Identifier_DRB24  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRB25  Item_Identifier_DRB48  Item_Identifier_DRC01  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRC12  Item_Identifier_DRC13  Item_Identifier_DRC25  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRC27  Item_Identifier_DRC36  Item_Identifier_DRC49  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRD01  Item_Identifier_DRD12  Item_Identifier_DRD13  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0.0                    0.0                    0.0   

      Item_Identifier_DRD15  Item_Identifier_DRD24  Item_Identifier_DRD25  \
1921                    0.0                    0.0                    0.0   
655                     0.0                    0.0                    0.0   
2229                    0.0                    0.0                    0.0   
2537                    0.0                    0.0                    0.0   
3528                    0


shape of processed data is: (5966, 1580)

There are 0 missing values

The datatypes are Item_Weight                      float64
Item_Visibility                  float64
Item_MRP                         float64
Outlet_Establishment_Year        float64
Outlet_Size                      float64
                                  ...   
Outlet_Identifier_OUT049         float64
Outlet_Type_Grocery Store        float64
Outlet_Type_Supermarket Type1    float64
Outlet_Type_Supermarket Type2    float64
Outlet_Type_Supermarket Type3    float64
Length: 1580, dtype: object
